# Day 16: Reindeer Maze

[Day 16](https://adventofcode.com/2024/day/16)

## Part 1

In [35]:
def load_grid(file_path):
    with open(file_path, 'r') as f:
        grid = [list(line.strip()) for line in f]
        return grid

grid = load_grid("small_input.txt")

def find_starting_position(grid):
    for row in range(len(grid)):
        for col in range(len(grid[0])):
            if grid[row][col] == 'S':
                return (row, col)

In [ ]:


from heapq import heappop, heappush

def heuristic(a, b):
    # Manhattan distance heuristic
    return abs(a[0] - b[0]) + abs(a[1] - b[1])

def a_star(grid, start, goal_char='E'):
    rows, cols = len(grid), len(grid[0])
    
    open_set = []
    heappush(open_set, (0, start))
    came_from = {}
    g_score = {start: 0}
    f_score = {start: 0}  # Start has no heuristic score initially

    while open_set:
        _, current = heappop(open_set)
        
        # Check if we've reached the goal
        if grid[current[0]][current[1]] == goal_char:
            path = []
            while current in came_from:
                path.append(current)
                current = came_from[current]
            path.append(start)
            return path[::-1]

        # Explore neighbors
        for dx, dy in [(-1, 0), (1, 0), (0, -1), (0, 1)]:
            neighbor = (current[0] + dx, current[1] + dy)

            # Ensure neighbor is within bounds
            if 0 <= neighbor[0] < rows and 0 <= neighbor[1] < cols:
                # Neighbor must be passable or the goal
                if grid[neighbor[0]][neighbor[1]] in ('.', goal_char):
                    tentative_g_score = g_score[current] + 1
                    if neighbor not in g_score or tentative_g_score < g_score[neighbor]:
                        came_from[neighbor] = current
                        g_score[neighbor] = tentative_g_score
                        f_score[neighbor] = tentative_g_score + heuristic(neighbor, (0, 0))
                        heappush(open_set, (f_score[neighbor], neighbor))
    
    print("No path found.")
    return None


path = run_a_star(grid, find_starting_position(grid))

print(path)

print(calculate_score(path))

[(139, 1), (139, 2), (139, 3), (138, 3), (137, 3), (136, 3), (135, 3), (134, 3), (133, 3), (132, 3), (131, 3), (130, 3), (129, 3), (128, 3), (127, 3), (127, 4), (127, 5), (128, 5), (129, 5), (129, 6), (129, 7), (129, 8), (129, 9), (129, 10), (129, 11), (128, 11), (127, 11), (126, 11), (125, 11), (125, 12), (125, 13), (125, 14), (125, 15), (124, 15), (123, 15), (123, 16), (123, 17), (123, 18), (123, 19), (124, 19), (125, 19), (125, 20), (125, 21), (125, 22), (125, 23), (124, 23), (123, 23), (123, 22), (123, 21), (122, 21), (121, 21), (121, 22), (121, 23), (121, 24), (121, 25), (120, 25), (119, 25), (119, 24), (119, 23), (118, 23), (117, 23), (116, 23), (115, 23), (114, 23), (113, 23), (113, 24), (113, 25), (113, 26), (113, 27), (112, 27), (111, 27), (111, 28), (111, 29), (111, 30), (111, 31), (110, 31), (109, 31), (108, 31), (107, 31), (107, 32), (107, 33), (106, 33), (105, 33), (105, 32), (105, 31), (104, 31), (103, 31), (102, 31), (101, 31), (101, 30), (101, 29), (100, 29), (99, 29), 

In [ ]:
import sys

sys.setrecursionlimit(10000)


def find_all_paths(grid, start, goal_char='E'):
    rows, cols = len(grid), len(grid[0])
    all_paths = []

    def dfs(path, current):
        x, y = current

        # Base case: if we've reached the goal, save the path
        if grid[x][y] == goal_char:
            all_paths.append(path[:])  # Add a copy of the path
            return

        # Explore neighbors
        for dx, dy in [(-1, 0), (1, 0), (0, -1), (0, 1)]:
            nx, ny = x + dx, y + dy

            # Ensure neighbor is within bounds and valid
            if 0 <= nx < rows and 0 <= ny < cols and grid[nx][ny] in ('.', goal_char):
                next_cell = (nx, ny)
                if next_cell not in path:  # Avoid revisiting cells in the current path
                    path.append(next_cell)
                    dfs(path, next_cell)
                    path.pop()  # Backtrack

    # Start the search
    dfs([start], start)

    return all_paths

def run_dfs(grid):
    for row in range(len(grid)):
        for col in range(len(grid[0])):
            if grid[row][col] == "S":
                return find_all_paths(grid, (row, col))

def calculate_score(path):
    cur_dir = 0
    score = 1
    for i in range(1, len(path) - 1):
        if abs(path[i][0] - path[i-1][0]) == 1:
            if(cur_dir == 2):
                score += 1000
            score += 1
            cur_dir = 1 # up or down
        elif abs(path[i][1] - path[i-1][1]) == 1:
            if(cur_dir == 1):
                score += 1000
            score += 1
            cur_dir = 2 #left or right
    return score

grid = load_grid("input.txt")

path_scores = []

for path in run_dfs(grid):
    path_scores.append(calculate_score(path))

print(min(path_scores))

In [ ]:
import plotly.graph_objects as go
import numpy as np
import plotly.io as pio
pio.renderers.default = "browser"  # Set the default renderer to browser


start = find_starting_position(grid)
goal_char = 'E'
rows, cols = len(grid), len(grid[0])
visited_cells = []  # To keep track of visited cells during DFS
all_paths = []      # To store all valid paths to the goal

grid_array = np.array([[1 if cell == '#' else 0 for cell in row] for row in grid])

# Real-time DFS with visualization
def dfs_real_time(path, current, frames):
    x, y = current

    # Mark the current cell as visited for visualization
    visited_cells.append(current)
    frames.append(list(visited_cells))  # Add to frames for animation

    # Base case: if we've reached the goal
    if grid[x][y] == goal_char:
        return

    # Explore neighbors
    for dx, dy in [(-1, 0), (1, 0), (0, -1), (0, 1)]:
        nx, ny = x + dx, y + dy

        # Ensure neighbor is within bounds and valid
        if 0 <= nx < rows and 0 <= ny < cols and grid[nx][ny] in ('.', goal_char):
            next_cell = (nx, ny)
            if next_cell not in path:  # Avoid revisiting cells in the current path
                path.append(next_cell)

                # Update the grid visualization immediately
                frames[-1].append(next_cell)
                #print(f"Visiting: {next_cell}")
                
                # Recursive call
                dfs_real_time(path, next_cell, frames)
                path.pop()  # Backtrack

# Create Plotly grid visualization
def create_plot(frames):
    fig = go.Figure()

    # Initialize the grid (walls as black, free cells as white)
    fig.add_trace(go.Heatmap(z=grid_array, colorscale='Greys', showscale=False))

    # Initialize scatter plot for visited cells
    fig.add_trace(go.Scatter(
        x=[y for x, y in frames[0]],
        y=[x for x, y in frames[0]],
        mode='markers',
        marker=dict(size=12, color='blue')
    ))

    # Set layout for the grid and add axis labels
    fig.update_layout(
        title="DFS Search Visualization",
        xaxis=dict(title="X Position", range=[0, cols-1], zeroline=False),
        yaxis=dict(title="Y Position", range=[0, rows-1], zeroline=False),
        showlegend=False,
        sliders=[{
            'steps': [
                {
                    'method': 'animate',
                    'label': f"Step {i}",
                    'args': [[f"Step {i}"], {'frame': {'duration': 300, 'redraw': True}, 'mode': 'immediate'}]
                }
                for i in range(len(frames))
            ],
            'active': 0
        }],
        updatemenus=[{
            'type': 'buttons',
            'buttons': [{
                'label': 'Play',
                'method': 'animate',
                'args': [None, {'frame': {'duration': 500, 'redraw': True}, 'fromcurrent': True}]
            }]
        }]
    )

    # Add frames for animation (each frame contains visited cells)
    fig.frames = [go.Frame(
        data=[
            go.Scatter(
                x=[y for x, y in frame],
                y=[x for x, y in frame],
                mode="markers",
                marker=dict(size=12, color='blue')
            )
        ],
        name=f"Step {i}"
    ) for i, frame in enumerate(frames)]

    # Show the figure
    fig.show()

# Real-time DFS exploration with animation
def visualize_dfs_real_time():
    frames = []
    dfs_real_time([start], start, frames)
    create_plot(frames)

# Call the function to visualize the DFS search
visualize_dfs_real_time()